# Laboratório 3.5 - Notebook do aluno

## Visão geral

Este laboratório é uma continuação dos laboratórios guiados do Módulo 3. 

Neste laboratório, você implantará um modelo já treinado e executará uma previsão relativa a ele. Em seguida, você excluirá o endpoint e executará uma transformação de lote (batch) no conjunto de dados (dataset) de teste.


## Introdução ao cenário do negócio

Você trabalha para um provedor de serviços médicos e deseja melhorar a detecção de anormalidades em pacientes ortopédicos. 

Você tem a incumbência de resolver esse problema usando machine learning (ML). Você tem acesso a um conjunto de dados que contém seis componentes biomecânicos e um alvo (target) de *normal* (normal) ou *abnormal* (anormal). Você pode usar esse conjunto de dados (dataset) para treinar um modelo de ML para prever se um paciente terá uma anomalia.


## Sobre esse conjunto de dados (dataset)

Esse conjunto de dados (dataset) biomédicos foi criado pelo Dr. Henrique da Mota durante um período de residência médica no Group of Applied Research in Orthopaedics (GARO) do Centre Médico-Chirurgical de Réadaptation des Massues em Lyon, na França. Os dados foram organizados em duas tarefas de classificação diferentes, mas relacionadas. 

A primeira tarefa consiste em classificar os pacientes como pertencentes a uma das três categorias a seguir: 

- *Normal* (Normal) (100 pacientes)
- *Disk Hernia* (Hérnia de disco) (60 pacientes)
- *Spondylolisthesis* (Espondilolistese) (150 pacientes)

Para a segunda tarefa, as categorias *Disk Hernia* (Hérnia de disco) e *Spondylolisthesis* (Espondilolistese) foram mescladas em uma única categoria, rotulada como *abnormal* (anormal). Portanto, a segunda tarefa consiste em classificar os pacientes como pertencentes a uma das categorias: *Normal* (Normal) (100 pacientes) ou *Abnormal* (Anormal) (210 pacientes).


## Informações dos atributos

Cada paciente é representado no conjunto de dados (dataset) por seis atributos biomecânicos derivados da forma e da orientação da pelve e da coluna lombar (nesta ordem): 

- Incidência pélvica
- Inclinação pélvica
- Ângulo da lordose lombar
- Inclinação sacral
- Raio pélvico
- Grau de espondilolistese

A convenção a seguir é usada para os rótulos de classe (labels): 
- DH (hérnia de disco)
- Espondilolistese (SL)
- Normal (NO) 
- Anormal (AB)

Para obter mais informações sobre esse conjunto de dados (dataset), consulte a [página da Web Conjunto de dados de coluna vertebral](http://archive.ics.uci.edu/ml/datasets/Vertebral+Column).


## Atribuições do conjunto de dados (dataset)

Esse conjunto de dados (dataset) foi obtido de:
Dua, D. e Graff, C. (2019). repositório UCI Machine Learning (http://archive.ics.uci.edu/ml). Irvine, CA: University of California, School of Information and Computer Science.


# Configuração do laboratório

Como essa solução é dividida em vários laboratórios no módulo, você executará as seguintes células para poder carregar os dados e treinar o modelo a ser implantado.

**Observação:** A configuração pode demorar até 5 minutos para ser concluída.

## Importação de dados

Ao executar as células a seguir, os dados serão importados e estarão prontos para uso. 

**Observação:** As células a seguir representam as principais etapas dos laboratórios anteriores.


In [ ]:
bucket='c117523a2804660l6883286t1w729814179638-labbucket-s5gmtv0pvzp0'

In [ ]:
import warnings, requests, zipfile, io
warnings.simplefilter('ignore')
import pandas as pd
from scipy.io import arff

import os
import boto3
import sagemaker
from sagemaker.image_uris import retrieve
from sklearn.model_selection import train_test_split

In [ ]:
f_zip = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00212/vertebral_column_data.zip'
r = requests.get(f_zip, stream=True)
Vertebral_zip = zipfile.ZipFile(io.BytesIO(r.content))
Vertebral_zip.extractall()

data = arff.loadarff('column_2C_weka.arff')
df = pd.DataFrame(data[0])

class_mapper = {b'Abnormal':1,b'Normal':0}
df['class']=df['class'].replace(class_mapper)

cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]

train, test_and_validate = train_test_split(df, test_size=0.2, random_state=42, stratify=df['class'])
test, validate = train_test_split(test_and_validate, test_size=0.5, random_state=42, stratify=test_and_validate['class'])

prefix='lab3'

train_file='vertebral_train.csv'
test_file='vertebral_test.csv'
validate_file='vertebral_validate.csv'

s3_resource = boto3.Session().resource('s3')
def upload_s3_csv(filename, folder, dataframe):
    csv_buffer = io.StringIO()
    dataframe.to_csv(csv_buffer, header=False, index=False )
    s3_resource.Bucket(bucket).Object(os.path.join(prefix, folder, filename)).put(Body=csv_buffer.getvalue())

upload_s3_csv(train_file, 'train', train)
upload_s3_csv(test_file, 'test', test)
upload_s3_csv(validate_file, 'validate', validate)

container = retrieve('xgboost',boto3.Session().region_name,'1.0-1')

hyperparams={"num_round":"42",
             "eval_metric": "auc",
             "objective": "binary:logistic"}

s3_output_location="s3://{}/{}/output/".format(bucket,prefix)
xgb_model=sagemaker.estimator.Estimator(container,
                                       sagemaker.get_execution_role(),
                                       instance_count=1,
                                       instance_type='ml.m4.xlarge',
                                       output_path=s3_output_location,
                                        hyperparameters=hyperparams,
                                        sagemaker_session=sagemaker.Session())

train_channel = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/train/".format(bucket,prefix,train_file),
    content_type='text/csv')

validate_channel = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/validate/".format(bucket,prefix,validate_file),
    content_type='text/csv')

data_channels = {'train': train_channel, 'validation': validate_channel}

xgb_model.fit(inputs=data_channels, logs=False)

print('ready for hosting!')

# Etapa 1: Hospedagem do modelo

Agora que você tem um modelo treinado, pode hospedá-lo usando os serviços de hospedagem do Amazon SageMaker.

A primeira etapa é a implantação do modelo. Como você tem um objeto de modelo,*xgb_model*, pode usar o método **deploy** (implantar). Para este laboratório, você usará uma única instância ml.m4.xlarge.



In [ ]:
xgb_predictor = xgb_model.deploy(initial_instance_count=1,
                serializer = sagemaker.serializers.CSVSerializer(),
                instance_type='ml.m4.xlarge')

# Etapa 2: Executar predições

Agora que tem um modelo implantado, você executará algumas predições.

Primeiramente, analise os dados de teste e familiarize-se com eles novamente.

In [ ]:
test.shape

Você tem 31 instâncias, com sete atributos. As primeiras cinco instâncias são:

In [ ]:
test.head(5)

Você não precisa incluir o valor do alvo (classe). Esse preditor pode obter dados no formato CSV (valores separados por vírgula). Portanto, você pode obter a primeira linha *sem a coluna de classe* usando o seguinte código:

`test.iloc[:1,1:]` 

A função **iloc** usa parâmetros de [*rows*,*cols*]

Para obter apenas a primeira linha, use `0:1`. Se você deseja obter a linha 2, pode usar `1:2`.

Para obter todas as colunas, *exceto* a primeira coluna (*col 0*), use `1:`



In [ ]:
row = test.iloc[0:1,1:]
row.head()

Você pode converter isso em um arquivo CSV (valores separados por vírgula) e armazenar em um buffer de string.

In [ ]:
batch_X_csv_buffer = io.StringIO()
row.to_csv(batch_X_csv_buffer, header=False, index=False)
test_row = batch_X_csv_buffer.getvalue()
print(test_row)

Agora, você pode usar os dados para executar uma predição.

In [ ]:
xgb_predictor.content_type = 'text/csv'
xgb_predictor.predict(test_row)

O resultado obtido não é *0* nem *1*. Em vez disso, você obtém uma *pontuação de probabilidade* (probability score). Você pode aplicar alguma lógica condicional à pontuação de probabilidade (probability score) para determinar se a resposta deverá ser apresentada como 0 ou 1. Você trabalhará com esse processo quando fizer predições em lote.

Por ora, compare o resultado com os dados de teste.

In [ ]:
test.head(5)

**Pergunta:** A predição é precisa?

**Tarefa de desafio:** Atualize o código anterior para enviar a segunda linha do conjunto de dados (dataset). Essas previsões estão corretas? Experimente esta tarefa com algumas outras linhas.

Pode ser entediante enviar essas linhas uma de cada vez. Você pode elaborar uma função para enviar esses valores em um lote (batch), mas o SageMaker já tem um recurso de lote (batch). Você examinará esse recurso em seguida. No entanto, antes de fazer isso, você encerrará o modelo.

# Etapa 3: Encerrar o modelo implantado

Para excluir o endpoint, use a função **delete_endpoint** no preditor.

In [ ]:
xgb_predictor.delete_endpoint(delete_endpoint_config=True)

# Etapa 4: Realizar uma transformação em lote (batch)

Quando você está no ciclo de engenharia de treinamento-teste dos componentes (features), deseja testar seus conjuntos de holdout (holdout sets) ou teste (test sets) em comparação com o modelo. Depois, você pode usar esses resultados para calcular métricas. Você pode implantar um endpoint como fez antes, mas deve se lembrar de excluir o endpoint. No entanto, existe uma maneira mais eficiente.

Você pode usar o método transformer no modelo para obter um objeto transformer (transformer object). Em seguida, você pode usar o método transformer deste objeto para executar uma predição em todo o conjunto de dados de teste (test dataset). O SageMaker: 

- Gerará uma instância com o modelo
- Executará uma predição em todos os valores de entrada
- Gravará esses valores no Amazon Simple Storage Service (Amazon S3) 
- Finalmente, encerrará a instância

Você poderá começar transformando seus dados em um arquivo CSV, o qual o objeto transformer pode usar como entrada. Dessa vez, você usará **iloc** para obter todas as linhas e todas as colunas, *exceto* a primeira coluna.


In [ ]:
batch_X = test.iloc[:,1:];
batch_X.head()

Em seguida, grave seus dados em um arquivo CSV.

In [ ]:
batch_X_file='batch-in.csv'
upload_s3_csv(batch_X_file, 'batch-in', batch_X)

Por fim, antes de executar uma transformação, configure o transformer com o arquivo de entrada, o local de saída e o tipo de instância.

In [ ]:
batch_output = "s3://{}/{}/batch-out/".format(bucket,prefix)
batch_input = "s3://{}/{}/batch-in/{}".format(bucket,prefix,batch_X_file)

xgb_transformer = xgb_model.transformer(instance_count=1,
                                       instance_type='ml.m4.xlarge',
                                       strategy='MultiRecord',
                                       assemble_with='Line',
                                       output_path=batch_output)

xgb_transformer.transform(data=batch_input,
                         data_type='S3Prefix',
                         content_type='text/csv',
                         split_type='Line')
xgb_transformer.wait()

Após a conclusão da transformação, você poderá fazer o download dos resultados do Amazon S3 e compará-los com a entrada.

Primeiramente, faça o download da saída do Amazon S3 e carregue-a em um DataFrame do pandas.


In [ ]:
s3 = boto3.client('s3')
obj = s3.get_object(Bucket=bucket, Key="{}/batch-out/{}".format(prefix,'batch-in.csv.out'))
target_predicted = pd.read_csv(io.BytesIO(obj['Body'].read()),',',names=['class'])
target_predicted.head(5)

Você pode usar uma função para converter a probabilidade em *0* ou *1*.

A primeira saída da tabela serão os *predicted values* (valores previstos), e a segunda saída da tabela serão os *original test data* (dados de teste originais).

In [ ]:
def binary_convert(x):
    threshold = 0.65
    if x > threshold:
        return 1
    else:
        return 0

target_predicted['binary'] = target_predicted['class'].apply(binary_convert)

print(target_predicted.head(10))
test.head(10)

**Observação:** O *limite* da função **binary_convert** é definido como *0,65*.

**Tarefa desafio:** experimente alterar o valor do limite (threshold). Isso afeta os resultados?

**Observação:** O modelo inicial pode não ser bom. Você gerará algumas métricas no próximo laboratório antes de ajustar o modelo no laboratório final.

# Parabéns!

Você concluiu este laboratório e agora pode encerrá-lo seguindo as instruções do guia do laboratório.